In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

In [8]:
def status_map(status):
    if 'no_relapse' in status or 'NoRelapse' in status:
        return 0
    else:
        return 1

class BcData:
    def __init__(self):
        self.data = pd.read_csv("data/data_good.csv")
        self.total = pd.read_csv("data/Total_old.csv", names=["gsm", "status"])
        self._drop_grey()

    # Drop grey columns
    def _drop_grey(self):
        status_list =['relapse', 'no_relapse', 'test1relapse',
                  'test1no_relapse', 'test2relapse',
                  'test2no_relapse', 'NewTest1_Relapse',
                  'NewTest1_NoRelapse', 'NewTest2_Relapse', 'NewTest2_NoRelapse']
        self.gsm_series = self.total[self.total.status.isin(status_list)].gsm
        new_cols = pd.Series(["GeneSymbol"]).append(self.gsm_series)

        self.total = self.total[self.total.gsm.isin(self.gsm_series)]
        self.data = self.data.filter(items=new_cols)

    # Group rows by gene leaving max median row
    def _groupby_gene(self):
        # Get max median in a group
        def _get_max_median(group):
            idx = group.median(axis=1).idxmax()
            return group.loc[idx]

        return self.data\
            .groupby("GeneSymbol", as_index=False, sort=False)\
            .apply(_get_max_median)

    def _log_table(self):
        self.data = np.log2(self.data)

    def _get_status(self):
        return self.total.status.map(status_map)

    # Drop rows with quantile less than threshold (values = {7, 8, 9})
    def filter_percentile(self, quantile=1, threshold=9):
        q = self.data.quantile(q=quantile, axis=1)
        index = q[q >= threshold].index.values
        self.data = self.data.loc[index, :]

    # Drop rows with max/min diff less than threshold (values = {1.5, 2})
    def filter_diff_percentile(self, qmax=1, qmin=0, threshold=2):
        threshold = np.log2(threshold)
        max = self.data.quantile(q=qmax, axis=1)
        min = self.data.quantile(q=qmin, axis=1)
        index = max[max - min >= threshold].index.values
        self.data = self.data.loc[index, :]

    # Get (X, y)
    def get_data(self):
        return self._groupby_gene(), self._get_status()

In [9]:
df = BcData()
df.data

,GeneSymbol,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,GSM441663,GSM441672,GSM441677,GSM441689,...,GSM79316,GSM79301,GSM79303,GSM79278,GSM79158,GSM79256,GSM79307,GSM79194,GSM79179,GSM79182
0,STAT1,271.7030,143.8760,96.2013,211.1580,245.9540,227.1580,369.5200,289.5180,137.6400,...,155.0600,139.5640,173.0080,183.1150,159.9650,122.8550,134.6120,157.5450,129.4740,124.1000
1,STAT1,148.6620,81.2455,58.1097,135.2550,169.8620,148.2000,190.4340,144.6220,88.7613,...,85.0733,86.2349,104.7460,60.2855,95.5286,58.8385,66.8661,74.5329,68.6508,69.4676
2,STAT1,562.0690,349.9580,191.2970,879.0090,771.7310,1017.0800,985.2080,730.3540,435.2680,...,485.3350,335.5130,686.7610,287.5500,754.0570,264.5480,246.7240,195.2160,245.2250,440.0420
3,GAPDH,4570.1500,2186.3100,4799.3800,6296.8700,4864.7700,3227.5400,4956.9400,4993.3600,3509.2400,...,2632.7900,2159.2200,3398.4900,1108.8800,1744.3200,1778.3400,1707.9800,1045.2000,2752.3600,1839.2100
4,GAPDH,4677.3700,2771.9800,5587.2800,7326.0300,5456.9500,3495.8900,5479.1200,4882.8900,3785.1700,...,3625.7000,3060.1500,5238.4500,2016.6800,3021.6800,2409.5600,2181.3000,2785.2900,3935.0600,2991.3200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15549,NAA40,54.4646,50.2889,50.6439,50.2440,45.1267,36.5843,42.9146,67.8202,74.0205,...,59.2511,59.7173,82.1648,41.8699,69.3741,58.1110,78.4802,55.5876,42.6167,57.9080
15550,BTRC,52.5145,43.5372,64.3767,48.1749,41.5742,46.1108,57.5067,41.9006,45.1893,...,68.8718,49.3107,61.5488,68.4528,51.3720,62.1168,55.3434,70.1898,62.0527,57.5067
15551,TBX10,47.5438,43.9503,56.3981,57.5737,92.4918,46.7302,49.7785,40.8507,44.4658,...,42.9858,40.1002,35.8531,42.1160,40.3773,41.5986,42.0447,55.3962,46.3305,41.0901
15552,KCNE4,32.1067,356.4300,244.8000,50.3291,126.3150,114.7860,336.0880,33.4743,62.8083,...,49.9980,230.9910,45.8824,53.5128,105.7030,56.9028,90.1375,163.1580,179.0010,46.0703


In [10]:
X, y = df.get_data()
X

,GeneSymbol,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,GSM441663,GSM441672,GSM441677,GSM441689,...,GSM79316,GSM79301,GSM79303,GSM79278,GSM79158,GSM79256,GSM79307,GSM79194,GSM79179,GSM79182
0,STAT1,1111.9500,700.3230,327.3760,1563.1700,1711.5900,1777.1600,1491.6300,998.5040,788.1790,...,730.1850,564.9710,1394.5000,388.5310,1279.2800,494.9020,397.7710,300.3530,415.9440,652.7210
1,GAPDH,9226.0800,6382.3400,10450.0000,13756.8000,11279.5000,7235.6500,10087.0000,8516.9200,7371.1900,...,9041.8200,6329.7600,12994.3000,5294.5400,6743.9100,5912.7500,6204.9900,6762.1600,10580.1000,6796.6000
2,ACTB,8039.9700,7094.6800,8810.0100,9381.7700,9996.9200,7357.9500,8061.1100,8133.9600,8976.7900,...,14421.5000,11631.3000,15244.0000,10808.4000,15063.6000,10617.0000,11992.1000,12650.9000,13686.7000,9801.7600
3,PRPF8,279.7840,892.7620,614.6880,290.4670,413.9100,368.8420,769.1610,404.6150,483.8170,...,615.7220,635.5830,395.4080,399.7820,671.8000,589.7670,382.8660,684.8490,348.2450,756.2910
4,CAPNS1,1154.8800,1089.7100,2224.1900,1463.4400,837.3450,1941.2700,1295.1400,1624.8000,1435.7800,...,1056.5800,1438.1500,1118.9900,1050.1900,1427.5400,1585.4500,1486.3000,990.4060,1066.7200,1220.2900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10502,OR7E156P,92.5358,86.0676,94.5928,89.0994,63.3539,95.3777,88.8058,111.1780,81.8165,...,113.3670,88.7571,199.3210,106.5780,108.0690,124.9120,116.6410,124.3250,92.2668,114.4590
10503,ALS2CL,135.8580,122.7750,231.7670,179.1850,127.0200,120.7980,143.7650,170.3350,111.4620,...,197.9460,158.3730,202.4980,232.8110,193.9110,161.3900,183.8290,279.3240,177.2180,214.1660
10504,C4orf34,18.6950,22.8111,20.1587,19.5876,20.0265,21.2956,19.1105,18.5949,21.2486,...,28.9671,20.8904,17.6882,21.1995,17.1490,25.7579,20.3404,22.9909,21.6785,20.2061
10505,TBX10,47.5438,43.9503,56.3981,57.5737,92.4918,46.7302,49.7785,40.8507,44.4658,...,42.9858,40.1002,35.8531,42.1160,40.3773,41.5986,42.0447,55.3962,46.3305,41.0901


In [26]:
X.set_index(list(X.iloc[1:, :1]))

,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,GSM441663,GSM441672,GSM441677,GSM441689,GSM441690,...,GSM79316,GSM79301,GSM79303,GSM79278,GSM79158,GSM79256,GSM79307,GSM79194,GSM79179,GSM79182
GeneSymbol,,,,,,,,,,,,,,,,,,,,,
STAT1,1111.9500,700.3230,327.3760,1563.1700,1711.5900,1777.1600,1491.6300,998.5040,788.1790,2312.6900,...,730.1850,564.9710,1394.5000,388.5310,1279.2800,494.9020,397.7710,300.3530,415.9440,652.7210
GAPDH,9226.0800,6382.3400,10450.0000,13756.8000,11279.5000,7235.6500,10087.0000,8516.9200,7371.1900,11221.5000,...,9041.8200,6329.7600,12994.3000,5294.5400,6743.9100,5912.7500,6204.9900,6762.1600,10580.1000,6796.6000
ACTB,8039.9700,7094.6800,8810.0100,9381.7700,9996.9200,7357.9500,8061.1100,8133.9600,8976.7900,9832.4000,...,14421.5000,11631.3000,15244.0000,10808.4000,15063.6000,10617.0000,11992.1000,12650.9000,13686.7000,9801.7600
PRPF8,279.7840,892.7620,614.6880,290.4670,413.9100,368.8420,769.1610,404.6150,483.8170,304.2150,...,615.7220,635.5830,395.4080,399.7820,671.8000,589.7670,382.8660,684.8490,348.2450,756.2910
CAPNS1,1154.8800,1089.7100,2224.1900,1463.4400,837.3450,1941.2700,1295.1400,1624.8000,1435.7800,770.7710,...,1056.5800,1438.1500,1118.9900,1050.1900,1427.5400,1585.4500,1486.3000,990.4060,1066.7200,1220.2900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OR7E156P,92.5358,86.0676,94.5928,89.0994,63.3539,95.3777,88.8058,111.1780,81.8165,75.3172,...,113.3670,88.7571,199.3210,106.5780,108.0690,124.9120,116.6410,124.3250,92.2668,114.4590
ALS2CL,135.8580,122.7750,231.7670,179.1850,127.0200,120.7980,143.7650,170.3350,111.4620,178.3070,...,197.9460,158.3730,202.4980,232.8110,193.9110,161.3900,183.8290,279.3240,177.2180,214.1660
C4orf34,18.6950,22.8111,20.1587,19.5876,20.0265,21.2956,19.1105,18.5949,21.2486,21.4049,...,28.9671,20.8904,17.6882,21.1995,17.1490,25.7579,20.3404,22.9909,21.6785,20.2061


In [ ]:
# X = df.filter_percentile(quantile=1, threshold=9)
X = df.filter_diff_percentile(qmax=0.75, qmin=0.25, threshold=1.8)
y = df.get_status()
print("Number of features: {}".format(len(X.columns)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Set dual = True if number of features > number of examples and vice versa
clf = svm.LinearSVC(penalty='l1', dual=False, C=0.1, max_iter=10000)
# clf = svm.SVC(kernel='linear', C=1)

scores = cross_val_score(clf, X, y, cv=5)
scores


In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Set dual = True if number of features > number of examples and vice versa
clf = svm.LinearSVC(penalty='l1', dual=False, C=0.1, max_iter=10000)
# clf = svm.SVC(kernel='linear', C=1)

scores = cross_val_score(clf, X, y, cv=5)
scores


array([0.76666667, 0.73333333, 0.73333333, 0.72483221, 0.73154362])